# టెక్స్ట్ వర్గీకరణ పని

ఈ మాడ్యూల్‌లో, మనం సులభమైన టెక్స్ట్ వర్గీకరణ పనితో ప్రారంభిస్తాము, ఇది **[AG_NEWS](http://www.di.unipi.it/~gulli/AG_corpus_of_news_articles.html)** డేటాసెట్ ఆధారంగా ఉంటుంది: మనం వార్తా శీర్షికలను 4 వర్గాలలో ఒకటిగా వర్గీకరిస్తాము: వరల్డ్, స్పోర్ట్స్, బిజినెస్ మరియు సై/టెక్.

## డేటాసెట్

డేటాసెట్‌ను లోడ్ చేయడానికి, మనం **[TensorFlow Datasets](https://www.tensorflow.org/datasets)** APIని ఉపయోగిస్తాము.


In [1]:
import tensorflow as tf
from tensorflow import keras
import tensorflow_datasets as tfds

# In this tutorial, we will be training a lot of models. In order to use GPU memory cautiously,
# we will set tensorflow option to grow GPU memory allocation when required.
physical_devices = tf.config.list_physical_devices('GPU') 
if len(physical_devices)>0:
    tf.config.experimental.set_memory_growth(physical_devices[0], True)

dataset = tfds.load('ag_news_subset')

మేము ఇప్పుడు `dataset['train']` మరియు `dataset['test']` ఉపయోగించి డేటాసెట్ యొక్క శిక్షణ మరియు పరీక్ష భాగాలను వరుసగా యాక్సెస్ చేయవచ్చు:


In [3]:
ds_train = dataset['train']
ds_test = dataset['test']

print(f"Length of train dataset = {len(ds_train)}")
print(f"Length of test dataset = {len(ds_test)}")

Length of train dataset = 120000
Length of test dataset = 7600


మన డేటాసెట్ నుండి మొదటి 10 కొత్త శీర్షికలను ముద్రిద్దాం:


In [4]:
classes = ['World', 'Sports', 'Business', 'Sci/Tech']

for i,x in zip(range(5),ds_train):
    print(f"{x['label']} ({classes[x['label']]}) -> {x['title']} {x['description']}")

3 (Sci/Tech) -> b'AMD Debuts Dual-Core Opteron Processor' b'AMD #39;s new dual-core Opteron chip is designed mainly for corporate computing applications, including databases, Web services, and financial transactions.'
1 (Sports) -> b"Wood's Suspension Upheld (Reuters)" b'Reuters - Major League Baseball\\Monday announced a decision on the appeal filed by Chicago Cubs\\pitcher Kerry Wood regarding a suspension stemming from an\\incident earlier this season.'
2 (Business) -> b'Bush reform may have blue states seeing red' b'President Bush #39;s  quot;revenue-neutral quot; tax reform needs losers to balance its winners, and people claiming the federal deduction for state and local taxes may be in administration planners #39; sights, news reports say.'
3 (Sci/Tech) -> b"'Halt science decline in schools'" b'Britain will run out of leading scientists unless science education is improved, says Professor Colin Pillinger.'
1 (Sports) -> b'Gerrard leaves practice' b'London, England (Sports Network

## టెక్స్ట్ వెక్టరైజేషన్

ఇప్పుడు టెక్స్ట్‌ను టెన్సార్లుగా ప్రాతినిధ్యం వహించగల **సంఖ్యలుగా** మార్చాలి. మనం పద స్థాయి ప్రాతినిధ్యం కావాలంటే, రెండు పనులు చేయాలి:

* టెక్స్ట్‌ను **టోకెన్లుగా** విభజించడానికి **టోకనైజర్** ఉపయోగించాలి.
* ఆ టోకెన్ల యొక్క **వోకాబ్యులరీ**ని నిర్మించాలి.

### వోకాబ్యులరీ పరిమితి

AG News డేటాసెట్ ఉదాహరణలో, వోకాబ్యులరీ పరిమాణం చాలా పెద్దది, 100k కంటే ఎక్కువ పదాలు ఉన్నాయి. సాధారణంగా, టెక్స్ట్‌లో అరుదుగా కనిపించే పదాలు అవసరం ఉండవు — కొన్ని వాక్యాల్లో మాత్రమే ఉంటాయి, మరియు మోడల్ వాటి నుండి నేర్చుకోదు. అందుకే, వోకాబ్యులరీ పరిమాణాన్ని చిన్న సంఖ్యకు పరిమితం చేయడం మంచిది, vectorizer కన్‌స్ట్రక్టర్‌కు ఆర్గ్యుమెంట్ ఇచ్చి:

ఈ రెండు దశలను **TextVectorization** లేయర్ ఉపయోగించి నిర్వహించవచ్చు. ఇప్పుడు vectorizer ఆబ్జెక్ట్‌ను సృష్టించి, ఆపై `adapt` మెథడ్‌ను పిలిచి అన్ని టెక్స్ట్‌ను గమనించి వోకాబ్యులరీని నిర్మిద్దాం:


In [5]:
vocab_size = 50000
vectorizer = keras.layers.experimental.preprocessing.TextVectorization(max_tokens=vocab_size)
vectorizer.adapt(ds_train.take(500).map(lambda x: x['title']+' '+x['description']))

> **గమనిక** మేము మొత్తం డేటాసెట్‌లోని ఒక ఉపసమితిని మాత్రమే వాక్యకోశం తయారుచేసేందుకు ఉపయోగిస్తున్నాము. ఇది అమలు సమయాన్ని వేగవంతం చేయడానికి మరియు మీకు వేచి ఉండకుండా చేయడానికి చేస్తాము. అయితే, మొత్తం డేటాసెట్‌లోని కొన్ని పదాలు వాక్యకోశంలో చేర్చబడకపోవచ్చు మరియు శిక్షణ సమయంలో అవి పరిగణించబడవు అనే ప్రమాదం తీసుకుంటున్నాము. కాబట్టి, మొత్తం వాక్యకోశ పరిమాణాన్ని ఉపయోగించి `adapt` సమయంలో మొత్తం డేటాసెట్ ద్వారా వెళ్లడం తుది ఖచ్చితత్వాన్ని పెంచుతుంది, కానీ గణనీయంగా కాదు.

ఇప్పుడు మేము వాస్తవ వాక్యకోశాన్ని యాక్సెస్ చేయవచ్చు:


In [6]:
vocab = vectorizer.get_vocabulary()
vocab_size = len(vocab)
print(vocab[:10])
print(f"Length of vocabulary: {vocab_size}")

['', '[UNK]', 'the', 'to', 'a', 'in', 'of', 'and', 'on', 'for']
Length of vocabulary: 5335


వెక్టర్ైజర్ ఉపయోగించి, మనం సులభంగా ఏదైనా టెక్స్ట్‌ను సంఖ్యల సెట్‌గా ఎన్‌కోడ్ చేయవచ్చు:


In [7]:
vectorizer('I love to play with my words')

<tf.Tensor: shape=(7,), dtype=int64, numpy=array([ 112, 3695,    3,  304,   11, 1041,    1], dtype=int64)>

## పదాల సంచయం టెక్స్ట్ ప్రాతినిధ్యం

పదాలు అర్థాన్ని సూచిస్తాయి కాబట్టి, వాక్యంలో వాటి క్రమం పక్కన పెట్టి, ఒక్కొక్క పదాలను చూసి కూడా ఒక టెక్స్ట్ భాగం అర్థం ఏమిటో అర్థం చేసుకోవచ్చు. ఉదాహరణకు, వార్తలను వర్గీకరించేటప్పుడు, *వాతావరణం* మరియు *మంచు* వంటి పదాలు *వాతావరణ సూచన*కి సంకేతం ఇవ్వగలవు, మరొకవైపు *స్టాక్స్* మరియు *డాలర్* వంటి పదాలు *ఆర్థిక వార్తలు*కి చెందుతాయి.

**పదాల సంచయం** (BoW) వెక్టర్ ప్రాతినిధ్యం అనేది అతి సులభంగా అర్థం చేసుకునే సంప్రదాయ వెక్టర్ ప్రాతినిధ్యం. ప్రతి పదం ఒక వెక్టర్ సూచికతో అనుసంధానించబడుతుంది, మరియు ఒక వెక్టర్ అంశం ఒక నిర్దిష్ట డాక్యుమెంట్‌లో ప్రతి పదం ఎన్ని సార్లు వచ్చిందో చూపిస్తుంది.

![పదాల సంచయం వెక్టర్ ప్రాతినిధ్యం మెమరీలో ఎలా ప్రాతినిధ్యం పొందుతుందో చూపించే చిత్రం.](../../../../../translated_images/te/bag-of-words-example.606fc1738f1d7ba9.webp) 

> **గమనిక**: BoWని టెక్స్ట్‌లోని ఒక్కొక్క పదానికి సంబంధించిన ఒక-హాట్-ఎన్‌కోడ్ చేసిన వెక్టర్ల మొత్తం అని కూడా భావించవచ్చు.

క్రింద Scikit Learn పైథాన్ లైబ్రరీ ఉపయోగించి పదాల సంచయం ప్రాతినిధ్యం ఎలా సృష్టించాలో ఒక ఉదాహరణ ఉంది:


In [8]:
from sklearn.feature_extraction.text import CountVectorizer
sc_vectorizer = CountVectorizer()
corpus = [
        'I like hot dogs.',
        'The dog ran fast.',
        'Its hot outside.',
    ]
sc_vectorizer.fit_transform(corpus)
sc_vectorizer.transform(['My dog likes hot dogs on a hot day.']).toarray()

array([[1, 1, 0, 2, 0, 0, 0, 0, 0]], dtype=int64)

మేము పైగా నిర్వచించిన Keras వెక్టరైజర్‌ను కూడా ఉపయోగించవచ్చు, ప్రతి పద సంఖ్యను ఒక-హాట్ ఎంకోడింగ్‌గా మార్చి ఆ వెక్టర్లన్నింటినీ కలిపి జోడించడం:


In [9]:
def to_bow(text):
    return tf.reduce_sum(tf.one_hot(vectorizer(text),vocab_size),axis=0)

to_bow('My dog likes hot dogs on a hot day.').numpy()

array([0., 5., 0., ..., 0., 0., 0.], dtype=float32)

> **గమనిక**: ఫలితం మునుపటి ఉదాహరణతో భిన్నంగా ఉండటం మీరు ఆశ్చర్యపోవచ్చు. కారణం ఏమిటంటే, Keras ఉదాహరణలో వెక్టర్ పొడవు మొత్తం AG News డేటాసెట్ నుండి నిర్మించిన పదసంపద పరిమాణానికి సరిపోతుంది, కానీ Scikit Learn ఉదాహరణలో మేము నమూనా టెక్స్ట్ నుండి తక్షణమే పదసంపదను నిర్మించాము.


## BoW వర్గీకర్తను శిక్షణ ఇవ్వడం

ఇప్పుడు మనం మన టెక్స్ట్ యొక్క బ్యాగ్-ఆఫ్-వర్డ్స్ ప్రాతినిధ్యాన్ని ఎలా నిర్మించాలో నేర్చుకున్నాము, దాన్ని ఉపయోగించే వర్గీకర్తను శిక్షణ ఇవ్వండి. మొదట, మన డేటాసెట్‌ను బ్యాగ్-ఆఫ్-వర్డ్స్ ప్రాతినిధ్యంగా మార్చాలి. ఇది క్రింది విధంగా `map` ఫంక్షన్ ఉపయోగించి సాధించవచ్చు:


In [11]:
batch_size = 128

ds_train_bow = ds_train.map(lambda x: (to_bow(x['title']+x['description']),x['label'])).batch(batch_size)
ds_test_bow = ds_test.map(lambda x: (to_bow(x['title']+x['description']),x['label'])).batch(batch_size)

ఇప్పుడు ఒక సాదా క్లాసిఫయర్ న్యూరల్ నెట్‌వర్క్‌ను నిర్వచిద్దాం, ఇది ఒక లీనియర్ లేయర్‌ను కలిగి ఉంటుంది. ఇన్‌పుట్ సైజ్ `vocab_size` మరియు అవుట్‌పుట్ సైజ్ క్లాసుల సంఖ్య (4) కు సరిపోతుంది. మనం ఒక క్లాసిఫికేషన్ టాస్క్‌ను పరిష్కరిస్తున్నందున, చివరి యాక్టివేషన్ ఫంక్షన్ **softmax** అవుతుంది:


In [12]:
model = keras.models.Sequential([
    keras.layers.Dense(4,activation='softmax',input_shape=(vocab_size,))
])
model.compile(loss='sparse_categorical_crossentropy',optimizer='adam',metrics=['acc'])
model.fit(ds_train_bow,validation_data=ds_test_bow)

938/938 [==============================] - 66s 70ms/step - loss: 0.6144 - acc: 0.8427 - val_loss: 0.4416 - val_acc: 0.8697


మనం 4 తరగతులు ఉన్నందున, 80% కంటే ఎక్కువ ఖచ్చితత్వం మంచి ఫలితం.

## ఒక నెట్‌వర్క్‌గా క్లాసిఫయర్‌ను శిక్షణ ఇవ్వడం

వెక్టర్‌యిజర్ కూడా Keras లేయర్ కావడంతో, దాన్ని కూడా కలిపి ఒక నెట్‌వర్క్‌ను నిర్వచించి, ఎండ్-టు-ఎండ్‌గా శిక్షణ ఇవ్వవచ్చు. ఈ విధంగా, డేటాసెట్‌ను `map` ఉపయోగించి వెక్టరైజ్ చేయాల్సిన అవసరం లేదు, మేము నెట్‌వర్క్ ఇన్‌పుట్‌కు అసలు డేటాసెట్‌ను నేరుగా ఇవ్వవచ్చు.

> **Note**: మేము ఇంకా మా డేటాసెట్‌పై మ్యాప్స్‌ను వర్తింపజేయాల్సి ఉంటుంది, ఫీల్డ్స్‌ను డిక్షనరీల నుండి (ఉదాహరణకు `title`, `description` మరియు `label`) టుపుల్స్‌గా మార్చడానికి. అయితే, డిస్క్ నుండి డేటాను లోడ్ చేసేటప్పుడు, అవసరమైన నిర్మాణంతో డేటాసెట్‌ను మొదటినుంచి నిర్మించవచ్చు.


In [13]:
def extract_text(x):
    return x['title']+' '+x['description']

def tupelize(x):
    return (extract_text(x),x['label'])

inp = keras.Input(shape=(1,),dtype=tf.string)
x = vectorizer(inp)
x = tf.reduce_sum(tf.one_hot(x,vocab_size),axis=1)
out = keras.layers.Dense(4,activation='softmax')(x)
model = keras.models.Model(inp,out)
model.summary()

model.compile(loss='sparse_categorical_crossentropy',optimizer='adam',metrics=['acc'])
model.fit(ds_train.map(tupelize).batch(batch_size),validation_data=ds_test.map(tupelize).batch(batch_size))


Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 1)]               0         
                                                                 
 text_vectorization (TextVec  (None, None)             0         
 torization)                                                     
                                                                 
 tf.one_hot (TFOpLambda)     (None, None, 5335)        0         
                                                                 
 tf.math.reduce_sum (TFOpLam  (None, 5335)             0         
 bda)                                                            
                                                                 
 dense_2 (Dense)             (None, 4)                 21344     
                                                                 
Total params: 21,344
Trainable params: 21,344
Non-trainable p

## బిగ్రామ్స్, ట్రైగ్రామ్స్ మరియు ఎన్-గ్రామ్స్

బ్యాగ్-ఆఫ్-వర్డ్స్ పద్ధతికి ఒక పరిమితి ఏమిటంటే, కొన్ని పదాలు బహుళ-పద వ్యక్తీకరణల భాగంగా ఉంటాయి, ఉదాహరణకు, 'హాట్ డాగ్' అనే పదానికి 'హాట్' మరియు 'డాగ్' అనే పదాల నుండి పూర్తిగా వేరే అర్థం ఉంటుంది. 'హాట్' మరియు 'డాగ్' అనే పదాలను ఎప్పుడూ ఒకే వెక్టర్లతో ప్రాతినిధ్యం చేస్తే, అది మన మోడల్‌ను గందరగోళం చేయవచ్చు.

దీనిని పరిష్కరించడానికి, **ఎన్-గ్రామ్ ప్రాతినిధ్యాలు** డాక్యుమెంట్ వర్గీకరణ పద్ధతుల్లో తరచుగా ఉపయోగిస్తారు, ఇక్కడ ప్రతి పదం, ద్విపదం లేదా త్రిపదం యొక్క సాంద్రత వర్గీకరణ శిక్షణకు ఉపయోగకరమైన లక్షణంగా ఉంటుంది. ఉదాహరణకు, బిగ్రామ్ ప్రాతినిధ్యాలలో, మేము అసలు పదాలతో పాటు అన్ని పద జంటలను కూడా పదసంపదలో చేర్చుతాము.

క్రింద స్కైకిట్ లెర్న్ ఉపయోగించి బిగ్రామ్ బ్యాగ్ ఆఫ్ వర్డ్ ప్రాతినిధ్యం ఎలా సృష్టించాలో ఒక ఉదాహరణ ఉంది:


In [14]:
bigram_vectorizer = CountVectorizer(ngram_range=(1, 2), token_pattern=r'\b\w+\b', min_df=1)
corpus = [
        'I like hot dogs.',
        'The dog ran fast.',
        'Its hot outside.',
    ]
bigram_vectorizer.fit_transform(corpus)
print("Vocabulary:\n",bigram_vectorizer.vocabulary_)
bigram_vectorizer.transform(['My dog likes hot dogs on a hot day.']).toarray()


Vocabulary:
 {'i': 7, 'like': 11, 'hot': 4, 'dogs': 2, 'i like': 8, 'like hot': 12, 'hot dogs': 5, 'the': 16, 'dog': 0, 'ran': 14, 'fast': 3, 'the dog': 17, 'dog ran': 1, 'ran fast': 15, 'its': 9, 'outside': 13, 'its hot': 10, 'hot outside': 6}


array([[1, 0, 1, 0, 2, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]],
      dtype=int64)

n-గ్రామ్ పద్ధతిలో ప్రధాన లోపం ఏమిటంటే, పదసంపద పరిమాణం చాలా వేగంగా పెరుగుతుంది. వాస్తవంలో, n-గ్రామ్ ప్రాతినిధ్యాన్ని *ఎంబెడ్డింగ్స్* వంటి కొలత తగ్గింపు సాంకేతికతతో కలపాలి, దీన్ని మేము తదుపరి యూనిట్‌లో చర్చించబోతున్నాము.

మా **AG News** డేటాసెట్‌లో n-గ్రామ్ ప్రాతినిధ్యాన్ని ఉపయోగించడానికి, `TextVectorization` కన్‌స్ట్రక్టర్‌కు `ngrams` పారామీటర్‌ను అందించాలి. బిగ్రామ్ పదసంపద పొడవు **గణనీయంగా ఎక్కువ** ఉంటుంది, మా సందర్భంలో ఇది 1.3 మిలియన్లకు పైగా టోకెన్లు! కాబట్టి, బిగ్రామ్ టోకెన్లను కూడా కొంత తగిన పరిమితితో నియంత్రించడం మంచిది.

పైన ఇచ్చిన కోడ్‌ను ఉపయోగించి క్లాసిఫయర్‌ను శిక్షణ ఇవ్వవచ్చు, కానీ అది చాలా మెమరీ-అసమర్థవంతంగా ఉంటుంది. తదుపరి యూనిట్‌లో, మేము ఎంబెడ్డింగ్స్ ఉపయోగించి బిగ్రామ్ క్లాసిఫయర్‌ను శిక్షణ ఇస్తాము. ఈ మధ్యలో, మీరు ఈ నోట్‌బుక్‌లో బిగ్రామ్ క్లాసిఫయర్ శిక్షణను ప్రయోగించి, ఎక్కువ ఖచ్చితత్వం సాధించగలరా అని చూడవచ్చు.


## ఆటోమేటిక్‌గా BoW వెక్టార్లను లెక్కించడం

పై ఉదాహరణలో మనం ఒక్కో పదం యొక్క వన్-హాట్ ఎంకోడింగ్స్‌ను జమచేసి చేతితో BoW వెక్టార్లను లెక్కించాము. అయితే, TensorFlow తాజా సంస్కరణలో, వెక్టరైజర్ కన్‌స్ట్రక్టర్‌కు `output_mode='count'` పారామీటర్‌ను అందించడం ద్వారా BoW వెక్టార్లను ఆటోమేటిక్‌గా లెక్కించవచ్చు. ఇది మన మోడల్ నిర్వచనం మరియు శిక్షణను గణనీయంగా సులభతరం చేస్తుంది:


In [15]:
model = keras.models.Sequential([
    keras.layers.experimental.preprocessing.TextVectorization(max_tokens=vocab_size,output_mode='count'),
    keras.layers.Dense(4,input_shape=(vocab_size,), activation='softmax')
])
print("Training vectorizer")
model.layers[0].adapt(ds_train.take(500).map(extract_text))
model.compile(loss='sparse_categorical_crossentropy',optimizer='adam',metrics=['acc'])
model.fit(ds_train.map(tupelize).batch(batch_size),validation_data=ds_test.map(tupelize).batch(batch_size))

Training vectorizer
938/938 [==============================] - 7s 7ms/step - loss: 0.5929 - acc: 0.8486 - val_loss: 0.4168 - val_acc: 0.8772


## పదం సాంద్రత - వ్యతిరేక డాక్యుమెంట్ సాంద్రత (TF-IDF)

BoW ప్రాతినిధ్యంలో, పదాల సంభవాలు పదం ఏదైనా ఉన్నా ఒకే విధంగా బరువు ఇవ్వబడతాయి. అయితే, *a* మరియు *in* వంటి తరచుగా వచ్చే పదాలు ప్రత్యేక పదాల కంటే వర్గీకరణకు తక్కువ ప్రాముఖ్యత కలిగి ఉంటాయని స్పష్టంగా తెలుస్తుంది. చాలా NLP పనుల్లో కొన్ని పదాలు ఇతరాల కంటే ఎక్కువ సంబంధితంగా ఉంటాయి.

**TF-IDF** అంటే **term frequency - inverse document frequency**. ఇది bag-of-words యొక్క ఒక రూపం, ఇందులో ఒక డాక్యుమెంట్‌లో పదం కనిపించిందని సూచించే 0/1 బైనరీ విలువ బదులు, పదం సంభవాల సాంద్రతకు సంబంధించిన ఫ్లోటింగ్-పాయింట్ విలువ ఉపయోగించబడుతుంది.

మరింత సాంప్రదాయంగా, డాక్యుమెంట్ $j$ లోని పదం $i$ యొక్క బరువు $w_{ij}$ ఈ విధంగా నిర్వచించబడుతుంది:
$$
w_{ij} = tf_{ij}\times\log({N\over df_i})
$$
ఇక్కడ
* $tf_{ij}$ అనేది $j$ లో $i$ పదం సంభవాల సంఖ్య, అంటే మునుపటి BoW విలువ
* $N$ అనేది సేకరణలో డాక్యుమెంట్ల సంఖ్య
* $df_i$ అనేది మొత్తం సేకరణలో $i$ పదం ఉన్న డాక్యుమెంట్ల సంఖ్య

TF-IDF విలువ $w_{ij}$ ఒక పదం డాక్యుమెంట్‌లో ఎంతసార్లు కనిపిస్తుందో అనుపాతంగా పెరుగుతుంది మరియు ఆ పదం ఉన్న డాక్యుమెంట్ల సంఖ్యతో తగ్గుతుంది, ఇది కొన్ని పదాలు ఇతరాల కంటే ఎక్కువగా కనిపించే వాస్తవాన్ని సరిచేస్తుంది. ఉదాహరణకు, ఒక పదం సేకరణలో *ప్రతి* డాక్యుమెంట్‌లో కనిపిస్తే, $df_i=N$, మరియు $w_{ij}=0$ అవుతుంది, అప్పుడు ఆ పదాలను పూర్తిగా పరిగణించరు.

Scikit Learn ఉపయోగించి మీరు సులభంగా టెక్స్ట్ యొక్క TF-IDF వెక్టరైజేషన్ సృష్టించవచ్చు:


In [16]:
from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer = TfidfVectorizer(ngram_range=(1,2))
vectorizer.fit_transform(corpus)
vectorizer.transform(['My dog likes hot dogs on a hot day.']).toarray()

array([[0.43381609, 0.        , 0.43381609, 0.        , 0.65985664,
        0.43381609, 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        ]])

Kerasలో, `TextVectorization` లేయర్ `output_mode='tf-idf'` పారామీటర్‌ను అందించడం ద్వారా ఆటోమేటిక్‌గా TF-IDF ఫ్రీక్వెన్సీలను గణించగలదు. TF-IDF ఉపయోగించడం accuracy పెరుగుతుందో లేదో చూడటానికి మనం పై కోడ్‌ను మళ్లీ రిపీట్ చేద్దాం:


In [17]:
model = keras.models.Sequential([
    keras.layers.experimental.preprocessing.TextVectorization(max_tokens=vocab_size,output_mode='tf-idf'),
    keras.layers.Dense(4,input_shape=(vocab_size,), activation='softmax')
])
print("Training vectorizer")
model.layers[0].adapt(ds_train.take(500).map(extract_text))
model.compile(loss='sparse_categorical_crossentropy',optimizer='adam',metrics=['acc'])
model.fit(ds_train.map(tupelize).batch(batch_size),validation_data=ds_test.map(tupelize).batch(batch_size))

Training vectorizer
938/938 [==============================] - 12s 12ms/step - loss: 0.4197 - acc: 0.8662 - val_loss: 0.3432 - val_acc: 0.8849


## ముగింపు

TF-IDF ప్రాతినిధ్యాలు వేర్వేరు పదాలకు తరచుదనం బరువులను ఇస్తున్నప్పటికీ, అవి అర్థం లేదా క్రమాన్ని ప్రతిబింబించలేవు. 1935లో ప్రసిద్ధ భాషావేత్త జె. ఆర్. ఫర్త్ చెప్పినట్లుగా, "ఒక పదం యొక్క పూర్తి అర్థం ఎప్పుడూ సందర్భానుసారంగా ఉంటుంది, మరియు సందర్భం లేకుండా అర్థం అధ్యయనం సీరియస్‌గా తీసుకోలేము." కోర్సులో తర్వాత భాషా మోడలింగ్ ఉపయోగించి టెక్స్ట్ నుండి సందర్భ సమాచారాన్ని ఎలా పొందాలో నేర్చుకుంటాం.


---

<!-- CO-OP TRANSLATOR DISCLAIMER START -->
**అస్పష్టత**:  
ఈ పత్రాన్ని AI అనువాద సేవ [Co-op Translator](https://github.com/Azure/co-op-translator) ఉపయోగించి అనువదించబడింది. మేము ఖచ్చితత్వానికి ప్రయత్నించినప్పటికీ, ఆటోమేటెడ్ అనువాదాల్లో పొరపాట్లు లేదా తప్పులుండవచ్చు. మూల పత్రం దాని స్వదేశీ భాషలో అధికారిక మూలంగా పరిగణించాలి. ముఖ్యమైన సమాచారానికి, ప్రొఫెషనల్ మానవ అనువాదం సిఫార్సు చేయబడుతుంది. ఈ అనువాదం వాడకంలో ఏర్పడిన ఏవైనా అపార్థాలు లేదా తప్పుదారితీసే అర్థాలు కోసం మేము బాధ్యత వహించము.
<!-- CO-OP TRANSLATOR DISCLAIMER END -->
